# Tasks
- used `kfoldStratified` splitting for this experiment _(`10 fold`)_.

- all the test are done inside the the loop of each fold to prevent data leakage and shuffle was `True`
    - Outliers more than `2xStandard Deviation` have been replaced with `mean`
    - removed `race` and `native-country`
    - Balance the dataset to directly by using `SmoteTomek` by keeping the ratio 1 for majority and minority class.
    - `MinMaxScaler` was used to scaling the train and test values
    - Null values for both train and test have been replaced with `mean` of train.
- Highest accuracy: `('LightGBM', 0.8592605017784137)`

# Import Libraries

In [1]:
import pandas as pd
import my_packages as mypckg
from collections import Counter
import numpy as np
from imblearn.combine import SMOTETomek
from sklearn.preprocessing import MinMaxScaler

%matplotlib inline

# Dataset

In [2]:
df = pd.read_csv('../data/adult.csv', na_values="?", skipinitialspace=True)

## Categorical Data Encoding

In [3]:
# encode the dataset
df , mappings = mypckg.encoder(df)

## Feature and Target Separation

In [4]:
X = df.drop(['race', 'native-country', 'income'], axis=1)
y = df['income']

# kfold

In [5]:
from sklearn.model_selection import StratifiedKFold
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

## Apply Models

In [6]:
models = mypckg.get_models()
models

{'Logistic Regression': LogisticRegression(max_iter=1000),
 'Decision Tree': DecisionTreeClassifier(),
 'Random Forest': RandomForestClassifier(),
 'Support Vector Classifier': SVC(),
 'Naive Bayes': GaussianNB(),
 'Gradient Boosting': GradientBoostingClassifier(),
 'AdaBoost': AdaBoostClassifier(),
 'LightGBM': LGBMClassifier(),
 'XGBoost': XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, device=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric='logloss',
               feature_types=None, gamma=None, grow_policy=None,
               importance_type=None, interaction_constraints=None,
               learning_rate=None, max_bin=None, max_cat_threshold=None,
               max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
               max_leaves=None, min_child_weight=None, missing=nan,
               monotone_constraints=None,

In [7]:
model_accuracies = {}
for name, model in models.items():
    fold_number = 1
    accuracies = []
    print(f"{'*' * 100}\n{'=' * 100}\nProcessing model: {name}\n{'=' * 100}\n{'*' * 100}")
    for train_index, test_index in kf.split(X, y):
        print(f"{'*' * 100}\n{':' * 100}\nModel Name: {name}\nFold # {fold_number}\n{':' * 100}\n{'*' * 100}")
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        if X_train.empty or X_test.empty:
            print("Empty train or test set encountered. Skipping this split.")
            continue


        X_train.fillna(X_train.mean(), inplace=True)
        X_test.fillna(X_train.mean(), inplace=True)

        y_train = y_train.loc[X_train.index]
        y_test= y_test.loc[X_test.index]

        for col in X_train.columns:
            mean = X_train[col].mean()
            sd = X_train[col].std()
            X_train[col].mask(X_train[col] > mean+(2*sd), mean, inplace=True)
            X_train[col].mask(X_train[col] < mean-(2*sd), mean, inplace=True)
        
        smote_tomek = SMOTETomek(sampling_strategy=1, random_state=42)
        X_train, y_train = smote_tomek.fit_resample(X_train, y_train)
        print(f"Y after resampling: {Counter(y_train)}")

        mm = MinMaxScaler()
        X_train_scaled_mm = mm.fit_transform(X_train)
        X_test_scaled_mm = mm.transform(X_test)

        model.fit(X_train_scaled_mm, y_train)
        y_pred = model.predict(X_test_scaled_mm)
        accuracies.append(mypckg.accuracy_score(y_test, y_pred))

        mypckg.print_evaluation_metrics(y_test, y_pred, model)
        fold_number += 1
    average_accuracy = np.mean(accuracies)
    print(f"Average Accuracy for {name}: {average_accuracy}\n\n")
    model_accuracies[name] = average_accuracy
print("All the accuracies:")
print("*" * 100)
print("*" * 100)
mypckg.show(model_accuracies)

****************************************************************************************************
Processing model: Logistic Regression
****************************************************************************************************
****************************************************************************************************
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
Model Name: Logistic Regression
Fold # 1
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
****************************************************************************************************


C:\Users\raqib\AppData\Local\Temp\ipykernel_9992\3288549735.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.fillna(X_train.mean(), inplace=True)
C:\Users\raqib\AppData\Local\Temp\ipykernel_9992\3288549735.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.fillna(X_train.mean(), inplace=True)
C:\Users\raqib\AppData\Local\Temp\ipykernel_9992\3288549735.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on whi

Y after resampling: Counter({0: 30825, 1: 30825})
Model: LogisticRegression(max_iter=1000)
Accuracy: 0.7488229273285568
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.75      0.82      3716
           1       0.48      0.76      0.59      1169

    accuracy                           0.75      4885
   macro avg       0.70      0.75      0.71      4885
weighted avg       0.81      0.75      0.76      4885

Confusion Matrix:
[[2770  946]
 [ 281  888]]
--------------------------------------------------------------------------------
****************************************************************************************************
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
Model Name: Logistic Regression
Fold # 2
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
************************************************************

C:\Users\raqib\AppData\Local\Temp\ipykernel_9992\3288549735.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.fillna(X_train.mean(), inplace=True)
C:\Users\raqib\AppData\Local\Temp\ipykernel_9992\3288549735.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.fillna(X_train.mean(), inplace=True)
C:\Users\raqib\AppData\Local\Temp\ipykernel_9992\3288549735.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on whi

Y after resampling: Counter({0: 30820, 1: 30820})
Model: LogisticRegression(max_iter=1000)
Accuracy: 0.7508700102354146
Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.74      0.82      3716
           1       0.49      0.77      0.60      1169

    accuracy                           0.75      4885
   macro avg       0.70      0.76      0.71      4885
weighted avg       0.81      0.75      0.77      4885

Confusion Matrix:
[[2763  953]
 [ 264  905]]
--------------------------------------------------------------------------------
****************************************************************************************************
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
Model Name: Logistic Regression
Fold # 3
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
************************************************************

...

Y after resampling: Counter({0: 30963, 1: 30963})
Model: BaggingClassifier()
Accuracy: 0.8044635544635544
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.87      0.87      3715
           1       0.59      0.60      0.59      1169

    accuracy                           0.80      4884
   macro avg       0.73      0.73      0.73      4884
weighted avg       0.80      0.80      0.80      4884

Confusion Matrix:
[[3233  482]
 [ 473  696]]
--------------------------------------------------------------------------------
****************************************************************************************************
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
Model Name: Bagging
Fold # 9
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
**************************************************************************************

C:\Users\raqib\AppData\Local\Temp\ipykernel_9992\3288549735.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.fillna(X_train.mean(), inplace=True)
C:\Users\raqib\AppData\Local\Temp\ipykernel_9992\3288549735.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.fillna(X_train.mean(), inplace=True)
C:\Users\raqib\AppData\Local\Temp\ipykernel_9992\3288549735.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on whi

Y after resampling: Counter({0: 30847, 1: 30847})
Model: BaggingClassifier()
Accuracy: 0.8005733005733006
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.86      0.87      3715
           1       0.58      0.62      0.60      1169

    accuracy                           0.80      4884
   macro avg       0.73      0.74      0.73      4884
weighted avg       0.81      0.80      0.80      4884

Confusion Matrix:
[[3180  535]
 [ 439  730]]
--------------------------------------------------------------------------------
****************************************************************************************************
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
Model Name: Bagging
Fold # 10
::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::::
*************************************************************************************

C:\Users\raqib\AppData\Local\Temp\ipykernel_9992\3288549735.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train.fillna(X_train.mean(), inplace=True)
C:\Users\raqib\AppData\Local\Temp\ipykernel_9992\3288549735.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test.fillna(X_train.mean(), inplace=True)
C:\Users\raqib\AppData\Local\Temp\ipykernel_9992\3288549735.py:25: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on whi

Y after resampling: Counter({0: 30877, 1: 30877})
Model: BaggingClassifier()
Accuracy: 0.8003685503685504
Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.87      0.87      3715
           1       0.58      0.58      0.58      1169

    accuracy                           0.80      4884
   macro avg       0.73      0.72      0.73      4884
weighted avg       0.80      0.80      0.80      4884

Confusion Matrix:
[[3231  484]
 [ 491  678]]
--------------------------------------------------------------------------------
Average Accuracy for Bagging: 0.8052293244207267


All the accuracies:
****************************************************************************************************
****************************************************************************************************
{
    "Logistic Regression": 0.7527334215205249,
    "Decision Tree": 0.7670652107397247,
    "Random Forest": 0.8342207169484549,
    "Support V

### Highest Accuracy

In [8]:
mypckg.highest_accuracy(model_accuracies)

The higest accuracy in this experiment:
LightGBM -->> 0.8592605017784137


('LightGBM', 0.8592605017784137)